In [124]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 7: Handy Haversacks ---</h2><p>You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to <em>issues in luggage processing</em>.</p>
<p>Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!</p>
<p>For example, consider the following rules:</p>
<pre><code>light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
</code></pre>
<p>These rules specify the required contents for 9 bag types. In this example, every <code>faded blue</code> bag is empty, every <code>vibrant plum</code> bag contains 11 bags (5 <code>faded blue</code> and 6 <code>dotted black</code>), and so on.</p>
<p>You have a <code><em>shiny gold</em></code> bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one <code>shiny gold</code> bag?)</p>
<p>In the above rules, the following options would be available to you:</p>
<ul>
<li>A <code>bright white</code> bag, which can hold your <code>shiny gold</code> bag directly.</li>
<li>A <code>muted yellow</code> bag, which can hold your <code>shiny gold</code> bag directly, plus some other bags.</li>
<li>A <code>dark orange</code> bag, which can hold <code>bright white</code> and <code>muted yellow</code> bags, either of which could then hold your <code>shiny gold</code> bag.</li>
<li>A <code>light red</code> bag, which can hold <code>bright white</code> and <code>muted yellow</code> bags, either of which could then hold your <code>shiny gold</code> bag.</li>
</ul>
<p>So, in this example, the number of bag colors that can eventually contain at least one <code>shiny gold</code> bag is <code><em>4</em></code>.</p>
<p><em>How many bag colors can eventually contain at least one <code>shiny gold</code> bag?</em> (The list of rules is quite long; make sure you get all of it.)</p>
</article>


In [125]:
example = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""


def parse_1(s):
    contained_by = defaultdict(set)

    for l in s.strip().splitlines():
        containing_bag, content = re.split(r"\s+bags\s+contain\s+", l)

        for contained in re.split(r",\s+", content):
            contained = re.sub(r"\s+bag(s?)\.?", "", contained)
            match = re.search(r"\d+", contained)

            if match:
                contained_bag = contained[match.end() + 1 :]
                contained_by[contained_bag].add(containing_bag)
    return contained_by


def how_many_bag_colors_can_eventually_contain_at_least_one_shiny_gold_bag(
    s: str,
) -> int:
    def dfs(bag):
        if bag in color_seen:
            return 0

        color_seen.add(bag)
        return sum((dfs(c) for c in contained_by.get(bag, [])), start=1)

    contained_by = parse_1(s)
    color_seen = set()

    return dfs("shiny gold") - 1


assert (
    how_many_bag_colors_can_eventually_contain_at_least_one_shiny_gold_bag(example) == 4
)

In [126]:
with open("../input/day7.txt") as f:
    puzzle = f.read()

print(
    f"Part I: {how_many_bag_colors_can_eventually_contain_at_least_one_shiny_gold_bag(puzzle)}"
)

Part I: 235


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>235</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!</p>
<p>Consider again your <code>shiny gold</code> bag and the rules from the above example:</p>
<ul>
<li><code>faded blue</code> bags contain <code>0</code> other bags.</li>
<li><code>dotted black</code> bags contain <code>0</code> other bags.</li>
<li><code>vibrant plum</code> bags contain <code>11</code> other bags: 5 <code>faded blue</code> bags and 6 <code>dotted black</code> bags.</li>
<li><code>dark olive</code> bags contain <code>7</code> other bags: 3 <code>faded blue</code> bags and 4 <code>dotted black</code> bags.</li>
</ul>
<p>So, a single <code>shiny gold</code> bag must contain 1 <code>dark olive</code> bag (and the 7 bags within it) plus 2 <code>vibrant plum</code> bags (and the 11 bags within <em>each</em> of those): <code>1 + 1*7 + 2 + 2*11</code> = <code><em>32</em></code> bags!</p>
<p>Of course, the actual rules have a <span title="100%">small</span> chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!</p>
<p>Here's another example:</p>
<pre><code>shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
</code></pre>
<p>In this example, a single <code>shiny gold</code> bag must contain <code><em>126</em></code> other bags.</p>
<p><em>How many individual bags are required inside your single <code>shiny gold</code> bag?</em></p>
</article>

</main>


In [127]:
example_2 = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
"""


def parse_2(s) -> dict[str, dict[str, int]]:
    contains = {}

    for l in s.strip().splitlines():
        containing_bag, content = re.split(r"\s+bags\s+contain\s+", l)
        contains[containing_bag] = {}

        for contained in re.split(r",\s+", content):
            contained = re.sub(r"\s+bag(s?)\.?", "", contained)
            match = re.search(r"\d+", contained)

            if match:
                n = int(contained[: match.end()])
                contained_bag = contained[match.end() + 1 :]
                contains[containing_bag][contained_bag] = n

    return contains


def count_bags_in_shiny_gold_bag(s: str) -> int:
    def dfs(bag):
        count = 0
        for b in contains.get(bag, []):
            count += contains[bag][b] * (1 + dfs(b))
        return count

    contains = parse_2(s)
    return dfs("shiny gold")


assert count_bags_in_shiny_gold_bag(example) == 32
assert count_bags_in_shiny_gold_bag(example_2) == 126

In [128]:
print(f"Part II: {count_bags_in_shiny_gold_bag(puzzle)}")

Part II: 158493


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>158493</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
